# Explainable Match API - Senseloaf
This interactive demo 🚀 will walk you through the process of using the Explainable Match API by SenseLoaf AI 🤖.

# Conventional matching vs Explainable Matching
Here, you might ask, what is an Explainable Match? 🤔
Let us first understand what Explainable Matching is, and how is it different (and better 📈) than conventional matching.

## Conventional Matching
The conventional document matching works on the context of the full document as a whole. In very simple terms, it will take the full documents as input and give out an overall match score. How does it match the documents? What kind of factors are considered while matching? 👀 Sadly, it works as a black-box, and we can not satisfactorily know that 😞

## Explainable Match
The explainable match solves the above problem of having a veil over the matching process, so that you know exactly how the matching is being done and what kind of factors is the model considering while matching, taking you from 😞 to 😁. 

## Senseloaf AI Explainable Match API
The Explainable Matching API by Senseloaf provides you with three different factors for matching:
- Matching by Experience 💼
- Matching by Job Titles 📋
- Matching by Skills 🧮

So, by using Senseloaf AI's Explainable match, along with the overall match score, you will also get the match score of the above factors individually, so you know, exactly how and what is being matched. Moreover, these three factors scores are merged to give you the overall score. And guess what? 🤔 You can even set the weightage of the individual factor scores (coming soon) to decide how much each score will contribute to the final overall score (you just went from 😁 to 🤩, didn't you?). 

# Using the Explainable Match API
For this demo, we will be matching a given Job Description document to a given Resume and find out how fit is a particular candidate for a job they have applied to, based on the three factors mentioned: Experience, Job Titles and Skills.

## Workflow of the Process
We will approach the process in the following manner:
1. Parse the resume using Senseloaf AI Resume Parser API
2. Parse the Job Description (called JD from now on) using the SenseLoaf AI JD Parser API
3. Send the result to the Senseloaf AI Explainable Match API

We at Senseloaf prefer Python as our programming language, and this demo is also formed in Python 🐍. 

# Creating your account 🔐
For using any APIs, you would first need an account on the Senseloaf Platform. If you do not already have an account, you can go ahead and create a free trial account on the Senseloaf Platform by visiting https://sandbox.senseloaf.ai

Once your account is created, you can continue with the demo. You will get some free trial credits as well to try out the products and use them in this demo as well.

# Logging into your account
Now that you have created our account, lets log into the account to get an Authorization Token. Authorization Token is used by our backend to identify the user. All our APIs use and require an authorization token to work.

In [14]:
import requests     # For making the API calls
import json         # For handling the responses

login_url = "https://service.senseloaf.com/login"          # Login API Endpoint
email = 'email@yourcompany.domain'                         # Email used to create the account
password = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXX'                 # Your password

payload = json.dumps({
    "emailId": email,
    "password": password
})
headers = {
    'Content-Type': 'application/json'
}

response = requests.request("POST", login_url, headers=headers, data=payload)
headers = response.headers        # Auth token is returned as header

auth_token = headers.get('Authorization')
auth_token          # This will be used for authorization

Bearer XXXXXXXXXXXXXXXXXXXXXXXXX


# Using the Senseloaf Resume Parser API
The Senseloaf resume parser API parses the given resume for the explainable AI match to select the fields from. For the demo, I have used a test resume which is of an NLP Engineer. You can find the file in ```resumes/test_resume.pdf```

In [2]:
resume_parser_url = 'https://service.senseloaf.com/api/v2/parse-resume'
test_resume_path = 'resumes/test_resume.pdf'
filename = 'test_resume.pdf'
payload = {}
files=[
    ('files',(filename, open(test_resume_path,'rb'), 'application/pdf'))
]
headers = {
    'Authorization': auth_token           # Token which we fetched using login API
}

resume = requests.request("POST", resume_parser_url, headers=headers, data=payload, files=files).json()
resume             # Parsed Resume

{'availableCredits': 8181,
 'errors': [],
 'number_of_resumes': 1,
 'result': [{'content': {'rawText': "contact (+91) 9773044674 rutviknitpy@gmail.com rutvik-trivedi education (May 2021) B.tech. in CSE, NATIONAL INSTITUTEOF TECHNOLOGY, Puducherry key skills Python Machine Learning. Natural Language Processing. Sklearn, Pandas, Numpy, NLTK, Keras, Pytorch, Gensim and other libraries Github C, Java, Octave (Beginner) Datastructures and Algorithms Web Development profile AMachine Learning and NLP enthusiast. Interested in Linguistics and its computation professional experience (February 2021 – Present) NLPENGINEER SENSELOAF TECHNOLOGIES – Bengaluru, IN ▪ One of themembers of the founding team.Working on building the next generation AI-based hiring platform.Worked on technologies and algorithms like AWS,MongoDB, Python, Gensim, Keras, BERT, Doc2Vec, Word2Vec, LDA (June 2020 – July 2020) Machine Learning Intern WIKILIMO Technologies – Remote ▪ Working on creatingML predictionmodels for vari

# Using the Senseloaf JD Parser API
The Senseloaf JD parser API parses the given JD and extract all the relevant fields from the JD like Job Title, Experience Level and Skills which will be used in turn by the Explainable Match API. The test JD used for this demo can be found at ```jds/jd.docx```

In [3]:
jd_parser_url = 'https://service.senseloaf.com/api/parse-jd'
jd_filename = 'jd.docx'        
jd_path = 'jds/jd.docx'

payload = {}
files=[
    ('files',(jd_filename, open(jd_path,'rb'), 'application/vnd.openxmlformats-officedocument.wordprocessingml.document'))
]
headers = {
    'Authorization': auth_token           # Token which we fetched using login API
}

jd = requests.request("POST", jd_parser_url, headers=headers, data=payload, files=files).json()
jd

{'availableCredits': 99990,
 'number_of_jd': 0,
 'result': [{'content': {'job_title': 'Sales Manager',
    'maximum_required_experience': 0,
    'minimum_required_experience': 10,
    'raw_text': "Ambarella, the worldwide leader in AI vision, HD and 4K video, is on a mission to bring AI and computer vision to all types of everyday devices, from surveillance cameras to robots to autonomous cars. In this position you will manage sales activities in the US including all video markets, with specific sales targets, including sales of Ambarella’s Emerging Market and IP Camera product lines. Key responsibilities: · Identify and drive design wins maximizing sales revenue from specific target accounts. · Maintain sales forecasting, deliver bi-weekly market and sales reports. · Drive design wins at key IP security camera customers, both professional and consumer. · Collaborate with marketing and engineering departments to help project management at key customers. · Work with Ambarella’s marketin

# Explainable Match API
Now that we got the Resume parsed as well as JD parsed, we can now send the parsed result to the explainable match API and see if I am eligible for the Job or not 😀

In [13]:
explainable_match_url = 'https://service.senseloaf.com/api/explainable-match'

payload = json.dumps({
    'jd': jd['result'][0],
    'resume': {
        'skills': resume['result'][0]['content']['skills'],
        'experience': resume['result'][0]['content']['sections'],
        'titles': resume['result'][0]['content']['experience']
    }
})
headers = {
    'Authorization': auth_token,           # Token which we fetched using login API
    'Content-Type': 'application/json'
}

explainable_match = requests.request("POST", explainable_match_url, headers=headers, data=payload).json()
explainable_match

{'result': {'missing_skills': ['project management',
   'ip',
   'target accounts',
   'ai',
   'security',
   'environment',
   'manage',
   'computer vision',
   'sales management',
   'vision',
   'managing',
   'marketing',
   'video',
   'leader',
   'design',
   'sales',
   'intellectual property management',
   'software',
   'sales forecasting',
   'go',
   'purchasing'],
  'scores': {'experience': {'grade': 'D', 'score': 0.22},
   'job_title': {'grade': 'E', 'score': 0.03},
   'overall_score': {'grade': 'E', 'score': 0.05},
   'skills': {'grade': 'E', 'score': 0.0}}},
 'availableCredits': 9998}

Huh, seems like the resume is not eligible for the job after all 😟. But that is correct. Why? Because if you see, the JD is of a Sr. Sales BDM and requires minimum 10 years of experience. While, being an NLP Engineer, the resume has no necessary skills for the JD. This can be clearly seen by the individual scores which are given by the explainable match, which is why explainable match is far more comprehensive than the conventional match 😇. You can even try and play around with any another JD or resume you like.

# Conclusion
That would conclude our demo for the explainable match API. I have tried to be as thorough as possible in the demo, but in case you are facing any difficulty using any of the APIs, we are always here to help, do not hesitate to reach out to us on team@senseloaf.com and we shall be happy to help you out with any problems you might be facing.

You can also refer to the API documentation for more advanced usage at https://docs.senseloaf.ai

**P.S.** : If you have any tips for the candidate to improve the score in the above JD, do reach out as well 😆